## **範例：多重 AI 代理人訂酒店**

在現今節奏急速的世界，計劃商務旅行不僅是訂機票和酒店。它需要一種協調與效率的水平，這往往難以達成。這正是多重 AI 代理人發揮作用的地方，顛覆我們管理旅遊需求的方式。

想像一下，你有一隊智能代理人為你效勞，協同處理旅行的各個方面，精確而輕鬆。憑藉我們先進的 AI 技術，我們創建了專門訂服務和編排行程的代理人，確保旅程無縫且無壓力。

這是一個基本場景。計劃商務旅行時，我們需諮詢商務旅行代理人以獲取機票資訊、酒店資訊等。透過 AI 代理人，我們可建立訂服務的代理人與編排行程的代理人，協作提升智慧水平。


# 初始化 Azure AI Agent Service 並從 **.env** 獲取配置信息

### **.env**

建立一個 .env 檔案

**.env** 包含 Azure AI Agent Service 的連接字串、AOAI 使用的模型，以及對應的 Google API 搜尋服務 API、ENDPOINT 等。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "你的 Azure AI Agent Service 模型部署名稱"

[**NOTE**] 你將需要一個具備每分鐘 100,000 令牌 (Tokens per minute) 速率限制和每分鐘 600 請求 (Request per minute) 速率限制的模型

  你可以在 Microsoft Foundry - 模型與端點中取得模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "你的 Azure AI Agent Service 專案連接字串"

  你可以在 AI Foundry 入口網站專案概覽頁面中取得專案連接字串。

- **SERPAPI_SEARCH_API_KEY** = "你的 SERPAPI 搜尋 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "你的 SERPAPI 搜尋端點"

要取得 Azure AI Agent Service 的模型部署名稱和專案連接字串，你需要先建立 Azure AI Agent Service。建議直接使用[此範本](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)進行建立（***注意:*** Azure AI Agent Service 目前只開放有限區域使用。建議參考[此連結](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) 來設定區域）

Agent 需要存取 SERPAPI。建議使用[此連結](https://serpapi.com/searches)註冊。註冊後，你可以獲得唯一的 API KEY 和 ENDPOINT。


# 登入 Azure

你現在需要登入 Azure，在 VScode 中打開終端機並執行 `az login` 命令


# 設定

要執行此筆記本，您需要安裝以下庫。以下是所需庫和對應的 pip 安裝命令列表：

azure-identity：用於 Azure 驗證。  
requests：用於發送 HTTP 請求。  
semantic-kernel：用於語義核心框架（假設這是一個自訂或特定的庫，您可能需要從特定來源或存儲庫安裝它）。


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# 解釋: 
import asyncio: 這會導入 asyncio 模組，該模組提供了 Python 中非同步編程的支援。它允許你使用 async 和 await 語法來編寫並發代碼。
from typing import Annotated: 這會從 typing 模組導入 Annotated 類型。Annotated 用於為類型提示添加元數據，這對於驗證、文檔或工具等多種用途非常有用。


In [ ]:
import asyncio,os
from typing import Annotated

# 解釋：
透過使用 from dotenv import load_dotenv 及 load_dotenv()，你可以輕鬆地在 .env 檔案中管理配置設定及敏感資訊（例如 API 金鑰和資料庫 URL），將它們與原始程式碼分開，使你的應用程式更安全且更易於配置。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 解釋：

匯入語句：from azure.identity.aio import DefaultAzureCredential：這會從 azure.identity.aio 模組中匯入 DefaultAzureCredential 類別。模組名稱中的 aio 部分表示它是設計用於非同步操作的。

DefaultAzureCredential 的用途：DefaultAzureCredential 類別是 Azure SDK for Python 的一部分。它提供了一種預設的方式來驗證 Azure 服務。它會依照特定順序嘗試使用多種方法進行驗證，例如環境變數、託管身份認證和 Azure CLI 認證。

非同步操作：aio 模組表示 DefaultAzureCredential 類別支援非同步操作。這意味著您可以使用 asyncio 來執行非阻塞的身份驗證請求。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 解釋：
從 semantic_kernel 套件中導入各種模組和類別。以下是每個導入項目的說明：

AgentGroupChat 來自 semantic_kernel.agents：這個類別處理與 AI 代理群組聊天相關的功能。AzureAIAgent 和 AzureAIAgentSettings 來自 semantic_kernel.agents.azure_ai

AzureAIAgent：這個類別用於建立和管理利用 Azure AI 服務的 AI 代理。

AzureAIAgentSettings：這個類別用於配置 AzureAIAgent 的設定。TerminationStrategy 來自 semantic_kernel.agents.strategies.termination.termination_strategy：

此類別定義在某些條件下終止 AI 代理執行的策略。ChatMessageContent 來自 semantic_kernel.contents.chat_message_content：

這個類別用於處理聊天訊息的內容。
AuthorRole 來自 semantic_kernel.contents.utils.author_role：

此類別定義在聊天訊息上下文中不同作者的角色。

kernel_function 來自 semantic_kernel.functions.kernel_function_decorator：這個裝飾器用於定義 kernel 函數，即可以在 semantic kernel 框架內執行的函數。
這些導入設定了建立和管理能在群組聊天環境中互動的 AI 代理所需的必要元件，可能用於例如訂房等任務。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explanation:
接著我們從 azure.ai.projects.models 模組導入 CodeInterpreterTool 類別。

CodeInterpreterTool：這個類別是 Azure AI SDK 的一部分，用於在 AI 項目的上下文中解釋和執行代碼。它提供執行代碼片段、分析代碼或將代碼執行整合到 AI 工作流程中的功能。
此導入設定了使用 CodeInterpreterTool 的必要組件，對於涉及動態解釋和執行代碼的任務可能非常有用。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explanation: 
ApprovalTerminationStrategy 類別提供了一種特定策略，用於終止 AI 代理的操作。當代理互動歷史中的最後一條訊息包含「saved」這個字時，代理將會終止。這在代理的任務被視為完成，並且收到已「saved」的確認時非常有用。定義互動方法。在預約計劃被保存後，當接收到 saved 訊號時，可以停止。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 解釋：

該行代碼透過呼叫 create() 方法，使用預設或預先定義的設定來初始化一個 AzureAIAgentSettings 物件。此設定物件（ai_agent_settings）隨後可用於配置和管理 AzureAIAgent 實例。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 說明：
透過引入 requests 函式庫，您可以輕鬆地在 Python 程式碼中發出 HTTP 請求並與網絡服務互動。


In [ ]:
import requests

# Explanation:
這是一個變量，用來存儲用於訪問 SERP（搜尋引擎結果頁）API 服務的 API 金鑰。API 金鑰是一個用於驗證與您的帳戶相關的請求的唯一標識符。

'GOOGLE_SEARCH_API_KEY'：這是一個佔位字串。您需要將 ''GOOGLE_SEARCH_API_KEY' 替換為您實際的 SERP API 金鑰。

用途：這行代碼的目的是將 API 金鑰存儲在變量中，以便用於驗證對 SERP API 服務的請求。存取服務及執行搜尋時需要這個 API 金鑰。

如何獲取 SERP API 金鑰：要獲得 SERP API 金鑰，請按照 https://serpapi.com 提供的一般步驟操作（具體步驟可能會根據您使用的 SERP API 服務有所不同）：

選擇 SERP API 服務：有多種 SERP API 服務可供選擇，例如 SerpAPI、Google 自訂搜尋 JSON API 及其他服務。請選擇最符合您需求的服務。

註冊帳戶：

前往所選 SERP API 服務的網站 https://www.serpapi.com 並註冊帳戶。您可能需要提供一些基本資料並驗證您的電子郵件地址。

建立 API 金鑰：

註冊後，登入您的帳戶並前往 API 部分或控制面板。尋找建立或產生新 API 金鑰的選項。

複製 API 金鑰：

API 金鑰產生後，請複製它。此金鑰將用於驗證您對 SERP API 服務的請求。

替換佔位符：

在您的 .env 檔案中替換該佔位符。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 解釋：
BookingPlugin 類別提供使用 Serpapi.com Google 搜尋 API 預訂酒店和航班的方法。它構建所需的參數，發送 API 請求，並處理回應以返回相關的預訂資訊。API 金鑰（SERPAPI_SEARCH_API_KEY）和端點（SERPAPI_SEARCH_ENDPOINT）用於驗證和發送請求到 Google 搜尋 API。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 說明：
SavePlugin 類別提供了一個方法 saving_plan，用以使用 Azure AI 服務儲存行程計劃。它設定 Azure 認證，建立 AI 代理，處理使用者輸入以產生並儲存行程計劃內容，並處理檔案儲存及清理操作。此方法於成功完成後會返回「Saved」。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 解釋：
此代碼設置了 Azure AI 代理，以處理航班和酒店預訂，並根據用戶輸入保存行程計劃。它使用 Azure 憑證來創建和配置代理，通過群聊處理用戶輸入，並確保在任務完成後進行適當的清理。代理使用特定插件（BookingPlugin 和 SavePlugin）來執行各自的任務。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們力求準確，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言文件應被視為權威來源。對於重要資訊，建議聘請專業人工翻譯。我們對因使用本翻譯所引起的任何誤解或誤釋概不負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
